In [1]:
# Make sure we can import contextualfairness
import sys

sys.path.append("../")

In [3]:
import pandas as pd

from contextualfairness.scorer import contextual_fairness_score
from contextualfairness.norms import BinaryClassificationEqualityNorm, RankNorm

In [4]:
# Setup fake data
ages = ["young", "young", "old", "young", "old"]
y_true = [1, 0, 0, 1, 0]

d = {
    "income": [50, 80, 30, 100, 30],
    "age": ages,
    "sex": ["male", "female", "male", "female", "male"],
    "y true": y_true,
}

X = pd.DataFrame(data=d)

In [5]:
# Define fake predictions and according probabilities
y_pred = [1, 0, 1, 0, 1]
y_pred_probas = [0.5, 0.2, 0.6, 0.4, 0.5]

In [6]:
# Define norm function


def richer_is_better(x):
    return x["income"]

In [9]:
norms = [BinaryClassificationEqualityNorm(0.5), RankNorm(0.5, richer_is_better)]

result = contextual_fairness_score(
    norms=norms,
    X=X,
    y_pred=y_pred,
    y_pred_probas=y_pred_probas,
)

/Users/pimk/Developer/contextual-fairness/examples/../contextualfairness/scorer.py:103: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.   0.25 0.   0.25 0.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[:, norm.name] = (
/Users/pimk/Developer/contextual-fairness/examples/../contextualfairness/scorer.py:103: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.05 0.15 0.   0.15 0.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[:, norm.name] = (


In [10]:
result.total_score()

np.float64(0.8500000000000001)

In [11]:
result.group_scores(["sex"])

{'sex=female': {'data': 1    0.4
  3    0.4
  Name: total, dtype: float64,
  'score': np.float64(0.8)},
 'sex=male': {'data': 0    0.05
  2    0.00
  4    0.00
  Name: total, dtype: float64,
  'score': np.float64(0.05)}}

In [13]:
result.group_scores(["sex", "age"], scaled=True)

/Users/pimk/Developer/contextual-fairness/examples/../contextualfairness/scorer.py:45: RuntimeWarning: invalid value encountered in scalar divide
  denominator += result[group_name]["score"] / len(
/Users/pimk/Developer/contextual-fairness/examples/../contextualfairness/scorer.py:51: RuntimeWarning: invalid value encountered in scalar divide
  (result[group_name]["score"] / len(result[group_name]["data"]))


{'sex=female;age=old': {'data': Series([], Name: total, dtype: float64),
  'score': np.float64(nan)},
 'sex=female;age=young': {'data': 1   NaN
  3   NaN
  Name: total, dtype: float64,
  'score': np.float64(nan)},
 'sex=male;age=old': {'data': 2   NaN
  4   NaN
  Name: total, dtype: float64,
  'score': np.float64(nan)},
 'sex=male;age=young': {'data': 0   NaN
  Name: total, dtype: float64,
  'score': np.float64(nan)}}